<a href="https://colab.research.google.com/github/tramyynt/Printer-Sale/blob/main/Excel%20file%20generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import calendar
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px

import sklearn
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,cross_val_score

from sklearn.metrics import r2_score
import sklearn.metrics as metrics

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/DataAnalystIntern_CaseStudy_Data.csv', delimiter = ';')

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848 entries, 0 to 847
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Customer ID           848 non-null    object
 1   Order ID              848 non-null    object
 2   Order Status          848 non-null    object
 3   Order Approval Date   848 non-null    object
 4   Order Delivered Date  401 non-null    object
 5   Product Code          848 non-null    object
 6   Product Name          848 non-null    object
 7   Product Quantity      848 non-null    object
 8   Product Ship Date     848 non-null    object
dtypes: object(9)
memory usage: 59.8+ KB


In [ ]:
data.isna().sum()

Customer ID               0
Order ID                  0
Order Status              0
Order Approval Date       0
Order Delivered Date    447
Product Code              0
Product Name              0
Product Quantity          0
Product Ship Date         0
dtype: int64

In [4]:
#convert string to int by replacing comma
data['Product Quantity'] = data['Product Quantity'].apply(lambda x: x.replace(',','.'))
data['Product Quantity'] = data['Product Quantity'].astype(float)
data['Product Quantity'] = data['Product Quantity'].astype(int)

In [5]:
data.head(5)

,Customer ID,Order ID,Order Status,Order Approval Date,Order Delivered Date,Product Code,Product Name,Product Quantity,Product Ship Date
0,11Y31JDd9,15d02hi1J,complete,31.12.2021,19.01.2022 15:40,FH-WA-01,Form Wash,1,13.01.2022 01:00
1,11Y31JDd9,15d02hi1J,complete,31.12.2021,19.01.2022 15:40,FH-CU-01,Form Cure,1,06.01.2022 01:00
2,11Y2uLTX8,15d02hhwE,complete,31.12.2021,19.01.2022 09:59,F3B-P-PRINTER,Form 3B+ 3D Printer,1,05.11.2025 01:00
3,11Y2uLTX8,15d02hhwE,complete,31.12.2021,19.01.2022 09:59,FH-WA-01,Form Wash,1,06.01.2022 01:00
4,11Y2uLTX8,15d02hhwE,complete,31.12.2021,19.01.2022 09:59,FH-CU-01,Form Cure,1,06.01.2022 01:00


In [8]:
#how many order each customer has and total unique products
Num_Ord = data.groupby('Customer ID')[['Order ID', 'Product Code']].nunique()
#data[data['Customer ID']=='1iqNlAT']
Num_Ord

,Order ID,Product Code
Customer ID,,
11Y2FaW5I,1,1
11Y2FakFP,1,3
11Y2FbEqW,1,1
11Y2FbLzm,1,2
11Y2FbMEp,1,1
...,...,...
1if2UoW,1,1
1iqNlAT,5,4
1iqNlIf,1,1


In [9]:
def record_cus (customer, df):
  record = df[df['Customer ID']== customer]
  return record

In [51]:
def get_index(arr, productId):
  index = np.where(arr == productId)
  return index[0][0]

In [70]:
"""
Save each customer with their orders with product quantity and status to dict
"""

list_customer = data['Customer ID'].unique()
list_product = data['Product Code'].unique()
a = {}
for i in list_customer:
  record = record_cus(i, data)
  ci = {}
  for j in range(len(record.index)):
    order_id = record.iloc[j,:]['Order ID']
    product_id = record.iloc[j,:]['Product Code']
    if not order_id in ci:
      ci[order_id] = np.empty(len(list_product), dtype=object)
    index = get_index(list_product,product_id)
    ci[order_id][index] = [record.iloc[j,:]['Product Name'],record.iloc[j,:]['Product Quantity'], record.iloc[j,:]['Order Status']]
  a[i] = ci


In [132]:
from sys import path
"""
Save to excel file
"""
import openpyxl

# Create the workbook and sheet for Excel
workbook = openpyxl.Workbook()
sheet = workbook.active

row = 1
column = 1
sheet.cell(row = row, column = column, value = "CustomerId")
column += 1
sheet.cell(row = row, column = column, value = "OrderId")
column += 1

for p in list_product:
  sheet.cell(row = row, column = column, value = p)
  column += 1

sheet.cell(row = row, column = column, value = "Status")
row += 1

for customerId in a.keys():
  # Put the key in the first column for each key in the dictionary
  sheet.cell(row=row, column=1, value=customerId)
  column = 2
  sum_arr = np.array([0]*len(list_product))
  for orderId in a[customerId]:
    #put order id in the first column
    sheet.cell(row = row, column = column, value = orderId)
    column = column +1

    s = "complete"
    arr = []

    for k in a[customerId][orderId]:
      if k == None:
        sheet.cell(row = row, column = column, value =0)
        arr.append(0)
      else:
        if k[2] != 'complete':
          s = "incomplete"
        sheet.cell(row = row, column = column, value = k[1])
        arr.append(k[1])
      column = column + 1

    sum_arr = sum_arr + arr
    sheet.cell(row = row, column = column, value = s)
    row = row +1
    column = 2

#calculate sum of each product for each customer
  column = 3
  for total in sum_arr:
    sheet.cell(row = row, column = column, value = total)
    column = column +1
  row = row +1

workbook.save(filename="my_workbook.xlsx")

In [130]:
ar = np.array([0]*len(list_product))

In [131]:
ar

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])